In [4]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy import stats

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams.update({'font.size': 13, 'figure.figsize': (8, 6)})
import seaborn as sns
sns.set_style('white')

In [5]:
df = pd.read_csv('datasets/data.csv')
df['time'] = df['time'].astype('timedelta64[h]')
df = df.set_index('time')

df2 = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

In [ ]:
df['e15x30SspGyrB'].resample('3h').mean().plot()

In [ ]:
cdata = df['e15x30SspGyrB'].resample('3h').mean()
cdata.tolist()

### Kinetics Model Writter

In [34]:
def parseUnidirection(reaction_id, reactants, products, rate, substrates={}):
    
    for r in reactants:
        name = str(r) + '_'
        output = '-1 * ' + str(rate)
        for r in reactants:
            output += ' * ' + r
        if name not in substrates:
            substrates[name] = output
        else:
            existing = substrates[name]
            substrates[name] = existing + ' + ' + output   
    if products[0] != '0':
        for p in products:
            name = str(p) + '_'
            output = str(rate)
            for r in reactants:
                output += ' * ' + r
            if name not in substrates:
                substrates[name] = output
            else:
                existing = substrates[name]
                substrates[name] = existing + ' + ' + output
    return substrates

def parseReaction(reaction_id, cr, substrates={}, rates=[]):
    
    direction = 'uni-direction' #default value is unidirection forward reaction
    cr_split_lead, cr_split_trail = cr.replace(' ', '').split('-')
    if cr_split_lead[-1]=='<' and cr_split_trail[0]=='>':
        direction = 'bi-direction'        
    reactants = cr_split_lead.replace('<', '').split('+')
    products = cr_split_trail.replace('>', '').split('+')   
    if direction=='uni-direction':
        rate = 'd' if products[0]=='0' else 'k'
        substrates = parseUnidirection(reaction_id, reactants, products, rate + reaction_id, substrates)
        if (rate + reaction_id) not in rates:
            rates.append(rate + reaction_id)
    else:
        substrates = parseUnidirection(reaction_id + 'f', reactants, products, 'f' + reaction_id, substrates)
        if ('f' + reaction_id) not in rates:
            rates.append('f' + reaction_id)
        substrates = parseUnidirection(reaction_id + 'b', products, reactants, 'r' + reaction_id, substrates)  
        if ('r' + reaction_id) not in rates:
            rates.append('r' + reaction_id)
    return substrates, rates

### AND Gate (Simple Model)

In [35]:
substrates = {}
rates = []
substrates, rates = parseReaction('pra', 'Ia+pA<->pAIa', substrates, rates)
#substrates, rates = parseReaction('rna', 'pAIa+RNAp<->pAIaRNAp', substrates, rates)
substrates, rates = parseReaction('ma', 'pAIa->pAIa+mA', substrates, rates)
substrates, rates = parseReaction('pa', 'mA->mA+A', substrates, rates)
substrates, rates = parseReaction('ma', 'mA->0', substrates, rates)
substrates, rates = parseReaction('a', 'A->0', substrates, rates)
#substrates, rates = parseReaction('ia', 'Ia->0', substrates, rates)

substrates, rates = parseReaction('prb', 'Ib+pB<->pBIb', substrates, rates)
#substrates, rates = parseReaction('rnb', 'pBIb+RNAp<->pBIbRNAp', substrates, rates)
substrates, rates = parseReaction('mb', 'pBIb->pBIb+mB', substrates, rates)
substrates, rates = parseReaction('pb', 'mB->mB+B', substrates, rates)
substrates, rates = parseReaction('mb', 'mB->0', substrates, rates)
substrates, rates = parseReaction('b', 'B->0', substrates, rates)
#substrates, rates = parseReaction('ib', 'Ib->0', substrates, rates)

substrates, rates = parseReaction('spl', 'A+B->Ic+w', substrates, rates)

substrates, rates = parseReaction('prc', 'Ic+pC<->pCIc', substrates, rates)
#substrates, rates = parseReaction('rnc', 'pCIc+RNAp<->pCIcRNAp', substrates, rates)
substrates, rates = parseReaction('mc', 'pCIc->pCIc+mC', substrates, rates)
substrates, rates = parseReaction('pc', 'mC->mC+C', substrates, rates)
substrates, rates = parseReaction('mc', 'mC->0', substrates, rates)
substrates, rates = parseReaction('c', 'C->0', substrates, rates)
substrates, rates = parseReaction('ic', 'Ic->0', substrates, rates)

In [36]:
with open('and_gate.py', 'w') as writer:
    writer.write('def andGate(y, t')
    for r in rates:
        writer.write(', ' + r)
    writer.write('):\n')
    writer.write('\t[')
    writer.write(', '.join([k[:-1] for k in substrates]) + '] = y\n')
    for k in substrates:
        writer.write('\t' + k + ' = ' + substrates[k] + '\n')
    writer.write('\treturn [' + ', '.join([k for k in substrates]) + ']')

In [ ]:
def andGate(y, t, fpra, rpra, kma, kpa, dma, da, fprb, rprb, kmb, kpb, dmb, db, kspl, fprc, rprc, kmc, kpc, dmc, dc, dic):
    [Ia, pA, pAIa, mA, A, Ib, pB, pBIb, mB, B, Ic, w, pC, pCIc, mC, C] = y
    Ia_ = -1 * fpra * Ia * pA + rpra * pAIa
    pA_ = -1 * fpra * Ia * pA + rpra * pAIa
    pAIa_ = fpra * Ia * pA + -1 * rpra * pAIa + -1 * kma * pAIa + kma * pAIa
    mA_ = kma * pAIa + -1 * kpa * mA + kpa * mA + -1 * dma * mA
    A_ = kpa * mA + -1 * da * A + -1 * kspl * A * B
    Ib_ = -1 * fprb * Ib * pB + rprb * pBIb
    pB_ = -1 * fprb * Ib * pB + rprb * pBIb
    pBIb_ = fprb * Ib * pB + -1 * rprb * pBIb + -1 * kmb * pBIb + kmb * pBIb
    mB_ = kmb * pBIb + -1 * kpb * mB + kpb * mB + -1 * dmb * mB
    B_ = kpb * mB + -1 * db * B + -1 * kspl * A * B
    Ic_ = kspl * A * B + -1 * fprc * Ic * pC + rprc * pCIc + -1 * dic * Ic
    w_ = kspl * A * B
    pC_ = -1 * fprc * Ic * pC + rprc * pCIc
    pCIc_ = fprc * Ic * pC + -1 * rprc * pCIc + -1 * kmc * pCIc + kmc * pCIc
    mC_ = kmc * pCIc + -1 * kpc * mC + kpc * mC + -1 * dmc * mC
    C_ = kpc * mC + -1 * dc * C
    return [Ia_, pA_, pAIa_, mA_, A_, Ib_, pB_, pBIb_, mB_, B_, Ic_, w_, pC_, pCIc_, mC_, C_]

t = np.linspace(0, 100)

rates = (0.5, 0.1, 0.5, 0.5, 0.1, 0.1,
         0.5, 0.1, 0.5, 0.5, 0.1, 0.1,
         0.8,
         0.5, 0.1, 0.5, 0.5, 0.1, 0.1,
         0.8)

y0 = [0.5, 0.8, 0.0, 0.0, 0.0,
      0.5, 0.8, 0.0, 0.0, 0.0,
      0.0, 0.0, 0.8, 0.0, 0.0, 0.0]

y = odeint(andGate, y0, t, rates)

plt.plot(t, y.transpose()[0], label='Ia')
plt.plot(t, y.transpose()[5], label='Ib')
plt.plot(t, y.transpose()[15], label='GFP')
plt.legend()
plt.show()

### AND Gate (Complex Model)

In [ ]:
substrates = {}
rates = []
substrates, rates = parseReaction('pra', 'Ia+pA<->pAIa', substrates, rates)
substrates, rates = parseReaction('rna', 'pAIa+RNAp<->pAIaRNAp', substrates, rates)
substrates, rates = parseReaction('ma', 'pAIaRNAp->pAIa+RNAp+mA', substrates, rates)
substrates, rates = parseReaction('pa', 'mA->mA+A', substrates, rates)
substrates, rates = parseReaction('ma', 'mA->0', substrates, rates)
substrates, rates = parseReaction('a', 'A->0', substrates, rates)
substrates, rates = parseReaction('ia', 'Ia->0', substrates, rates)

substrates, rates = parseReaction('prb', 'Ib+pB<->pBIb', substrates, rates)
substrates, rates = parseReaction('rnb', 'pBIb+RNAp<->pBIbRNAp', substrates, rates)
substrates, rates = parseReaction('mb', 'pBIbRNAp->pBIb+RNAp+mB', substrates, rates)
substrates, rates = parseReaction('pb', 'mB->mB+B', substrates, rates)
substrates, rates = parseReaction('mb', 'mB->0', substrates, rates)
substrates, rates = parseReaction('b', 'B->0', substrates, rates)
substrates, rates = parseReaction('ib', 'Ib->0', substrates, rates)

substrates, rates = parseReaction('spl1', 'A+B<->AxBx', substrates, rates)
substrates, rates = parseReaction('spl2', 'AxBx->Ic+w', substrates, rates)

substrates, rates = parseReaction('prc', 'Ic+pC<->pCIc', substrates, rates)
substrates, rates = parseReaction('rnc', 'pCIc+RNAp<->pCIcRNAp', substrates, rates)
substrates, rates = parseReaction('mc', 'pCIcRNAp->pCIc+RNAp+mC', substrates, rates)
substrates, rates = parseReaction('pc', 'mC->mC+C', substrates, rates)
substrates, rates = parseReaction('mc', 'mC->0', substrates, rates)
substrates, rates = parseReaction('c', 'C->0', substrates, rates)
substrates, rates = parseReaction('ic', 'Ic->0', substrates, rates)

In [ ]:
with open('and_gate.py', 'w') as writer:
    writer.write('def andGate(y, t')
    for r in rates:
        writer.write(', ' + r)
    writer.write('):\n')
    writer.write('\t[')
    writer.write(', '.join([k[:-1] for k in substrates]) + '] = y\n')
    for k in substrates:
        writer.write('\t' + k + ' = ' + substrates[k] + '\n')
    writer.write('\treturn [' + ', '.join([k for k in substrates]) + ']')

In [ ]:
from and_gate import andGate

t = np.linspace(0, 100)
rates = (0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.01,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.01, 0.8, 0.1, 0.7,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.01)

y0 = [0.5, 0.8, 0.0, 1.0, 0.0, 0.0, 0.0,
      0.5, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.0, 0.8, 0.0, 0.0, 0.0, 0.0]

y = odeint(andGate, y0, t, rates)

plt.plot(t, y[:,0], label='Ia')
plt.plot(t, y[:,7], label='Ib')
plt.plot(t, y[:,20], label='GFP')
plt.legend()
plt.show()

### XOR Gate

In [ ]:
substrates = {}
rates = []
substrates, rates = parseReaction('pra', 'Ia+pA<->pAIa', substrates, rates)
substrates, rates = parseReaction('rna', 'pAIa+RNAp<->pAIaRNAp', substrates, rates)
substrates, rates = parseReaction('ma', 'pAIaRNAp->pAIa+RNAp+mA+mIc', substrates, rates)
substrates, rates = parseReaction('pa', 'mA->mA+A', substrates, rates)
substrates, rates = parseReaction('pic', 'mIc->mIc+Ic', substrates, rates)
substrates, rates = parseReaction('ma', 'mA->0', substrates, rates)
substrates, rates = parseReaction('mic', 'mIc->0', substrates, rates)
substrates, rates = parseReaction('a', 'A->0', substrates, rates)
substrates, rates = parseReaction('ic', 'Ic->0', substrates, rates)
substrates, rates = parseReaction('ia', 'Ia->0', substrates, rates)

substrates, rates = parseReaction('prb', 'Ib+pB<->pBIb', substrates, rates)
substrates, rates = parseReaction('rnb', 'pBIb+RNAp<->pBIbRNAp', substrates, rates)
substrates, rates = parseReaction('mb', 'pBIbRNAp->pBIb+RNAp+mB+mIc', substrates, rates)
substrates, rates = parseReaction('pb', 'mB->mB+B', substrates, rates)
substrates, rates = parseReaction('pic', 'mIc->mIc+Ic', substrates, rates)
substrates, rates = parseReaction('mb', 'mB->0', substrates, rates)
substrates, rates = parseReaction('mic', 'mIc->0', substrates, rates)
substrates, rates = parseReaction('b', 'B->0', substrates, rates)
substrates, rates = parseReaction('ic', 'Ic->0', substrates, rates)
substrates, rates = parseReaction('ib', 'Ib->0', substrates, rates)

substrates, rates = parseReaction('spl1', 'A+B<->AxBx', substrates, rates)
substrates, rates = parseReaction('spl2', 'AxBx->AB+w', substrates, rates)

substrates, rates = parseReaction('ecf', 'AB+Ic<->ABIc', substrates, rates)
substrates, rates = parseReaction('ecf', 'ABIc->0', substrates, rates)

substrates, rates = parseReaction('prc', 'Ic+pC<->pCIc', substrates, rates)
substrates, rates = parseReaction('rnc', 'pCIc+RNAp<->pCIcRNAp', substrates, rates)
substrates, rates = parseReaction('mc', 'pCIcRNAp->pCIc+RNAp+mC', substrates, rates)
substrates, rates = parseReaction('pc', 'mC->mC+C', substrates, rates)
substrates, rates = parseReaction('mc', 'mC->0', substrates, rates)
substrates, rates = parseReaction('c', 'C->0', substrates, rates)
#substrates, rates = parseReaction('ic', 'Ic->0', substrates, rates)

#print(substrates.keys())
#for k in substrates:
#    print(k + ' = ' + substrates[k])
#print(list(set(rates)))

In [ ]:
with open('xor_gate.py', 'w') as writer:
    writer.write('def xorGate(y, t')
    for r in rates:
        writer.write(', ' + r)
    writer.write('):\n')
    writer.write('\t[')
    writer.write(', '.join([k[:-1] for k in substrates]) + '] = y\n')
    for k in substrates:
        writer.write('\t' + k + ' = ' + substrates[k] + '\n')
    writer.write('\treturn [' + ', '.join([k for k in substrates]) + ']')

In [ ]:
def xorGate2(y, t, fpra, rpra, frna, rrna, kma, kpa, kpic, dma, dmic, da, dia,
                    fprb, rprb, frnb, rrnb, kmb, kpb, dmb, db, dib,
                    fspl1, rspl1, kspl2, fecf, recf, decf,
                    fprc, rprc, frnc, rrnc, kmc, kpc, dmc, dc, dic):
    
    [Ia, pA, pAIa, RNAp, pAIaRNAp, mA, mIc, A, 
     Ib, pB, pBIb, pBIbRNAp, mB, B, AxBx, AB, w, ABIc,
     Ic, pC, pCIc, pCIcRNAp, mC, C] = y
    
    Ia_ = -1 * fpra * Ia * pA + rpra * pAIa + -1 * dia * Ia
    pA_ = -1 * fpra * Ia * pA + rpra * pAIa
    pAIa_ = fpra * Ia * pA + -1 * rpra * pAIa + -1 * frna * pAIa * RNAp + rrna * pAIaRNAp + kma * pAIaRNAp
    RNAp_ = -1 * frna * pAIa * RNAp + rrna * pAIaRNAp + kma * pAIaRNAp + -1 * frnb * pBIb * RNAp + rrnb * pBIbRNAp + kmb * pBIbRNAp + -1 * frnc * pCIc * RNAp + rrnc * pCIcRNAp + kmc * pCIcRNAp
    pAIaRNAp_ = frna * pAIa * RNAp + -1 * rrna * pAIaRNAp + -1 * kma * pAIaRNAp
    mA_ = kma * pAIaRNAp + -1 * kpa * mA + kpa * mA + -1 * dma * mA
    mIc_ = kma * pAIaRNAp + -1 * kpic * mIc + kpic * mIc + -1 * dmic * mIc + kmb * pBIbRNAp + -1 * kpic * mIc + kpic * mIc + -1 * dmic * mIc
    A_ = kpa * mA + -1 * da * A + -1 * fspl1 * A * B + rspl1 * AxBx
    Ic_ = kpic * mIc + -1 * dic * Ic + kpic * mIc + -1 * dic * Ic + -1 * fecf * AB * Ic + recf * ABIc + -1 * fprc * Ic * pC + rprc * pCIc
    Ib_ = -1 * fprb * Ib * pB + rprb * pBIb + -1 * dib * Ib
    pB_ = -1 * fprb * Ib * pB + rprb * pBIb
    pBIb_ = fprb * Ib * pB + -1 * rprb * pBIb + -1 * frnb * pBIb * RNAp + rrnb * pBIbRNAp + kmb * pBIbRNAp
    pBIbRNAp_ = frnb * pBIb * RNAp + -1 * rrnb * pBIbRNAp + -1 * kmb * pBIbRNAp
    mB_ = kmb * pBIbRNAp + -1 * kpb * mB + kpb * mB + -1 * dmb * mB
    B_ = kpb * mB + -1 * db * B + -1 * fspl1 * A * B + rspl1 * AxBx
    AxBx_ = fspl1 * A * B + -1 * rspl1 * AxBx + -1 * kspl2 * AxBx
    AB_ = kspl2 * AxBx + -1 * fecf * AB * Ic + recf * ABIc
    w_ = kspl2 * AxBx
    ABIc_ = fecf * AB * Ic + -1 * recf * ABIc + -1 * decf * ABIc
    pC_ = -1 * fprc * Ic * pC + rprc * pCIc
    pCIc_ = fprc * Ic * pC + -1 * rprc * pCIc + -1 * frnc * pCIc * RNAp + rrnc * pCIcRNAp + kmc * pCIcRNAp
    pCIcRNAp_ = frnc * pCIc * RNAp + -1 * rrnc * pCIcRNAp + -1 * kmc * pCIcRNAp
    mC_ = kmc * pCIcRNAp + -1 * kpc * mC + kpc * mC + -1 * dmc * mC
    C_ = kpc * mC + -1 * dc * C
    
    return [Ia_, pA_, pAIa_, RNAp_, pAIaRNAp_, mA_, mIc_, A_, 
            Ib_, pB_, pBIb_, pBIbRNAp_, mB_, B_, AxBx_, AB_, w_, ABIc_,
            Ic_, pC_, pCIc_, pCIcRNAp_, mC_, C_]

In [ ]:
from xor_gate import xorGate

t = np.linspace(0, 2000)
rates = (0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.001, 0.5, 0.2, 0.2, 1.5, 0.01,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.5, 0.2, 0.01,
         0.8, 0.001, 0.7, 20.0, 0.01, 10.0,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.2)

#fpra, rpra, frna, rrna, kma, kpa, kpic, dma, dmic, da, dia,
#fprb, rprb, frnb, rrnb, kmb, kpb, dmb, db, dib,
#fspl1, rspl1, kspl2, fecf, recf, decf,
#fprc, rprc, frnc, rrnc, kmc, kpc, dmc, dc, dic

y0 = [0.6, 0.8, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
      0.6, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.0, 0.8, 0.0, 0.0, 0.0, 0.0]

y = odeint(xorGate, y0, t, rates)

plt.plot(t, y.transpose()[0], label='Ia')
#plt.plot(t, y.transpose()[1], label='pA')
#plt.plot(t, y.transpose()[2], label='pAIa')
#plt.plot(t, y.transpose()[5], label='mA')
#plt.plot(t, y.transpose()[7], label='A')
#plt.plot(t, y.transpose()[12], label='mB')
#plt.plot(t, y.transpose()[13], label='B')

#plt.plot(t, y.transpose()[0], label='Ia')
#plt.plot(t, y.transpose()[6], label='mIc')
plt.plot(t, y.transpose()[8], label='Ib')
#plt.plot(t, y.transpose()[7], label='A')
#plt.plot(t, y.transpose()[13], label='B')
#plt.plot(t, y.transpose()[14], label='AxBx')
#plt.plot(t, y.transpose()[15], label='AB')
#plt.plot(t, y.transpose()[18], label='Ic')
plt.plot(t, y.transpose()[23], label='GFP')
plt.legend()
plt.show()

### Manually Designed Gates

In [ ]:
def andGate(y, t, kf1a, kb1a, kf2a, kb2a, kma, kpa, dma, da, dia,
                  kf1b, kb1b, kf2b, kb2b, kmb, kpb, dmb, db, dib,
                  kf1c, kb1c, kf2c, kb2c, kmc, kpc, dmc, dc, dic,
                  kf1s, kb1s, ks):
    
    [pA, Ia, pAIa, RNAp, pAIaRNAp, mA, A,
     pB, Ib, pBIb, pBIbRNAp, mB, B, AxBx, w,
     pC, Ic, pCIc, pCIcRNAp, mC, C] = y
    
    #input-1
    pA_ = kb1a * pAIa - kf1a * pA * Ia
    Ia_ = kb1a * pAIa - kf1a * pA * Ia - dia * Ia
    pAIa_ = kf1a * pA * Ia - kb1a * pAIa + kb2a * pAIaRNAp - kf2a * pAIa * RNAp + kma * pAIaRNAp
    #RNAp_ = kb2a * pAIaRNAp - kf2a * pAIa * RNAp + kma * pAIaRNAp
    RNAp_ = kb2a * pAIaRNAp - kf2a * pAIa * RNAp + kma * pAIaRNAp + kb2b * pBIbRNAp - kf2b * pBIb * RNAp + kmb * pBIbRNAp + kb2c * pCIcRNAp - kf2c * pCIc * RNAp + kmc * pCIcRNAp
    pAIaRNAp_ = kf2a * pAIa * RNAp - kb2a * pAIaRNAp - kma * pAIaRNAp
    mA_ = kma * pAIaRNAp - dma * mA
    A_ = kpa * mA - da * A + kb1s * AxBx - kf1s * A * B
    
    #input-2
    pB_ = kb1b * pBIb - kf1b * pB * Ib
    Ib_ = kb1b * pBIb - kf1b * pB * Ib - dib * Ib
    pBIb_ = kf1b * pB * Ib - kb1b * pBIb + kb2b * pBIbRNAp - kf2b * pBIb * RNAp + kmb * pBIbRNAp
    #RNAp_ = kb2b * pBIbRNAp - kf2b * pBIb * RNAp + kmb * pBIbRNAp
    pBIbRNAp_ = kf2b * pBIb * RNAp - kb2b * pBIbRNAp - kmb * pBIbRNAp
    mB_ = kmb * pBIbRNAp - dmb * mB
    B_ = kpb * mB - db * B + kb1s * AxBx - kf1s * A * B
    
    #split-intein
    #A_ = kb1s * AxBx - kf1s * A * B
    #B_ = kb1s * AxBx - kf1s * A * B
    AxBx_ = kf1s * A * B - kb1s * AxBx - ks * AxBx
    #Ic_ = ks * AxBx #output of protein splicing acts as inducer for output gate
    w_ = ks * AxBx
    
    #output
    pC_ = kb1c * pCIc - kf1c * pC * Ic
    Ic_ = kb1c * pCIc - kf1c * pC * Ic + ks * AxBx - dic * Ic
    pCIc_ = kf1c * pC * Ic - kb1c * pCIc + kb2c * pCIcRNAp - kf2c * pCIc * RNAp + kmc * pCIcRNAp
    #RNAp_ = kb2c * pCIcRNAp - kf2c * pCIc * RNAp + kmc * pCIcRNAp
    pCIcRNAp_ = kf2c * pCIc * RNAp - kb2c * pCIcRNAp - kmc * pCIcRNAp
    mC_ = kmc * pCIcRNAp - dmc * mC
    C_ = kpc * mC - dc * C
    
    return [pA_, Ia_, pAIa_, RNAp_, pAIaRNAp_, mA_, A_,
            pB_, Ib_, pBIb_, pBIbRNAp_, mB_, B_, AxBx_, w_,
            pC_, Ic_, pCIc_, pCIcRNAp_, mC_, C_]

In [ ]:
t = np.linspace(0, 50)
rates = (0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.1,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.1,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.1,
         0.8, 0.001, 0.7)
y0 = [0.6, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
      0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.5, 0.0, 0.0, 0.0, 0.0, 0.0]
y = odeint(andGate, y0, t, rates)

#plt.plot(t, y.transpose()[0], label='pA')
plt.plot(t, y.transpose()[1], label='Ia')
#plt.plot(t, y.transpose()[7], label='pB')
plt.plot(t, y.transpose()[8], label='Ib')
plt.plot(t, y.transpose()[20], label='GFP')
plt.legend()
plt.show()

In [ ]:
def xorGate(y, t, kf1a, kb1a, kf2a, kb2a, kma, kpa, dma, da, dia,
                  kf1b, kb1b, kf2b, kb2b, kmb, kpb, dmb, db, dib,
                  kf1c, kb1c, kf2c, kb2c, kmc, kpc, dmc, dc, dic,
                  kf1s, kb1s, ks, kf1ae, kb1ae, 
                  kpae, kpbe, dae, dbe,
                  kae, dmae, dmbe):
    
    [pA, Ia, pAIa, RNAp, pAIaRNAp, mA, A,
     pB, Ib, pBIb, pBIbRNAp, mB, B, AxBx, AB, w, ABIc, mIc,
     pC, Ic, pCIc, pCIcRNAp, mC, C] = y
    
    #input-1
    pA_ = kb1a * pAIa - kf1a * pA * Ia
    Ia_ = kb1a * pAIa - kf1a * pA * Ia - dia * Ia
    pAIa_ = kf1a * pA * Ia - kb1a * pAIa + kb2a * pAIaRNAp - kf2a * pAIa * RNAp + kma * pAIaRNAp
    #RNAp_ = kb2a * pAIaRNAp - kf2a * pAIa * RNAp + kma * pAIaRNAp
    RNAp_ = kb2a * pAIaRNAp - kf2a * pAIa * RNAp + kma * pAIaRNAp + kb2b * pBIbRNAp - kf2b * pBIb * RNAp + kmb * pBIbRNAp + kb2c * pCIcRNAp - kf2c * pCIc * RNAp + kmc * pCIcRNAp
    pAIaRNAp_ = kf2a * pAIa * RNAp - kb2a * pAIaRNAp - kma * pAIaRNAp
    mA_ = kma * pAIaRNAp - dma * mA
    mIc_ = kma * pAIaRNAp - dmae * mIc + kmb * pBIbRNAp - dmbe * mIc
    A_ = kpa * mA - da * A + kb1s * AxBx - kf1s * A * B
    Ic_ = kpae * mIc - dae * Ic + kb1ae * ABIc - kf1ae * AB * Ic + kpbe * mIc - dbe * Ic + kb1c * pCIc - kf1c * pC * Ic - dic * Ic + kb1ae * ABIc - kf1ae * AB * Ic
    
    #input-2
    pB_ = kb1b * pBIb - kf1b * pB * Ib
    Ib_ = kb1b * pBIb - kf1b * pB * Ib - dib * Ib
    pBIb_ = kf1b * pB * Ib - kb1b * pBIb + kb2b * pBIbRNAp - kf2b * pBIb * RNAp + kmb * pBIbRNAp
    #RNAp_ = kb2b * pBIbRNAp - kf2b * pBIb * RNAp + kmb * pBIbRNAp
    pBIbRNAp_ = kf2b * pBIb * RNAp - kb2b * pBIbRNAp - kmb * pBIbRNAp
    mB_ = kmb * pBIbRNAp - dmb * mB
    #mIc_ = kmb * pBIbRNAp - dmbe * mIc
    B_ = kpb * mB - db * B + kb1s * AxBx - kf1s * A * B
    #Ic_ = kpbe * mIc - dbe * Ic + kb1c * pCIc - kf1c * pC * Ic - dic * Ic
    
    #split-intein
    #A_ = kb1s * AxBx - kf1s * A * B
    #B_ = kb1s * AxBx - kf1s * A * B
    AxBx_ = kf1s * A * B - kb1s * AxBx - ks * AxBx
    AB_ = ks * AxBx +  2 * (kb1ae * ABIc - kf1ae * AB * Ic) #output of protein splicing acts as inducer for output gate
    w_ = ks * AxBx
    
    #ECFs
    #AB_ = kb1ae * ABIc - kf1ae * AB * Ic
    #Ic_ = kb1ae * ABIc - kf1ae * AB * Ic
    ABIc_ = kf1ae * AB * Ic - kb1ae * ABIc - kae * ABIc
    
    #output
    pC_ = kb1c * pCIc - kf1c * pC * Ic
    #Ic_ = kb1c * pCIc - kf1c * pC * Ic - dic * Ic
    pCIc_ = kf1c * pC * Ic - kb1c * pCIc + kb2c * pCIcRNAp - kf2c * pCIc * RNAp + kmc * pCIcRNAp
    #RNAp_ = kb2c * pCIcRNAp - kf2c * pCIc * RNAp + kmc * pCIcRNAp
    pCIcRNAp_ = kf2c * pCIc * RNAp - kb2c * pCIcRNAp - kmc * pCIcRNAp
    mC_ = kmc * pCIcRNAp - dmc * mC
    C_ = kpc * mC - dc * C
    
    return [pA_, Ia_, pAIa_, RNAp_, pAIaRNAp_, mA_, A_,
            pB_, Ib_, pBIb_, pBIbRNAp_, mB_, B_, AxBx_, AB_, w_, ABIc_, mIc_,
            pC_, Ic_, pCIc_, pCIcRNAp_, mC_, C_]

In [ ]:
t = np.linspace(0, 50)
rates = (0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.1,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.1,
         0.5, 0.001, 0.5, 0.05, 0.7, 0.8, 0.2, 0.4, 0.1,
         0.8, 0.001, 0.7, 0.8, 0.001,
         0.7, 0.7, 0.2, 0.2,
         0.7, 0.1, 0.1)
y0 = [0.5, 0.7, 0.0, 1.0, 0.0, 0.0, 0.0,
      0.6, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.5, 0.0, 0.0, 0.0, 0.0, 0.0]
y = odeint(xorGate, y0, t, rates)

#plt.plot(t, y.transpose()[0], label='pA')
plt.plot(t, y.transpose()[1], label='Ia')
#plt.plot(t, y.transpose()[7], label='pB')
plt.plot(t, y.transpose()[8], label='Ib')
plt.plot(t, y.transpose()[23], label='GFP')
plt.legend()
plt.show()